In [1]:
#no existe mas list len checker ni convert to float corregir calce_E_E

def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False        
def expand_dict_with_lists_inplace(input_dict):
    # Collect new key-value pairs
    new_items = {}
    
    # Loop through the original dictionary keys
    keys_to_remove = []  # To track which keys need to be removed
    
    for key, value in input_dict.items():
        # Check if the value is a list
        if isinstance(value, list):
            # Add the items in the list as new key-value pairs
            for index, item in enumerate(value):
                new_key = f"{key}_{index}"
                new_items[new_key] = item
            # Mark the original key for removal
            keys_to_remove.append(key)
    
    # Remove the original list-type keys
    for key in keys_to_remove:
        del input_dict[key]
    
    # Update the original dictionary with the new items
    input_dict.update(new_items)

def calc_e_e(dic):
    # Define the base key patterns to check for
    base_patterns=['MV Vel E','Med Vel E','Lat Vel E']
    # Dictionary to hold keys grouped by index (e.g., 0, 1, etc.)
    key_groups = {}

     # Outer loop: Group keys based on their index
    for key in dic.keys():
        # Check if the key matches one of the base patterns
        for pattern in base_patterns:
            if key.startswith(pattern) and not key.find('grad')>0:
                # Extract the index (assumes it's after the last underscore)
                try:
                    index = key.split('_')[-1]
                    # Ensure the index is treated as a string
                    if index.isdigit():
                        if index not in key_groups:
                            key_groups[index] = {}
                        key_groups[index][pattern] = key
                except IndexError:
                      pass
    e_e=[0,0]
    E_e_prom=[0,0]
    for index, keys in key_groups.items():
        # Check if all necessary keys are present for this index
        if all(base in keys for base in base_patterns):
            # Extract values from the dictionary
            mv_value = dic[keys['MV Vel E']]
            med_value = dic[keys['Med Vel E']]
            lat_value = dic[keys['Lat Vel E']]
            # Perform the calculation
            e_primas_prom = round((med_value+lat_value)/2,2)
            E_e_prom_value =round((mv_value)*100/e_primas_prom,2) if e_primas_prom else 0
            e_e[int(index)]=e_primas_prom
            E_e_prom[int(index)]=E_e_prom_value 
    return E_e_prom,e_e

def remove_signs(dic):
    
    dic_sin_signos={key.replace(' ', '_').replace('/','_').replace('(','_').replace(')','_').
                    replace('-','_').replace('%',''): value 
                    for key, value in dic.items()
                   }
    return dic_sin_signos

def convert_to_float(dic):
    key_to_round=['LVIDd','LVIDs','IVSd','%FS(2D)','LVd Mass Index(2D-ASE)'
                  ,'Ao Sinusus','Ao Diam','RAAd','RVAWd','LVPWd','LAVI'
                  ]
    for val in dic.keys():
        if 'Exam_Date' not in val:
            if isinstance(dic.get(val),list):
                valores=[]
                for i in dic.get(val):
                    if is_float(i):
                        valor=float(i.replace('-',''))
                        if val in key_to_round:
                            valor=int(round(valor,0))
                        valores.append(valor)
                dic[val]=valores
            else:
                value = dic.get(val)
                if isinstance(value, str) and is_float(value):
                    valor=float(value.replace('-', '')) 
                    if val in key_to_round:
                        dic[val]=int(round(valor,0))
                    else:          
                        dic[val] =valor
                    
    return dic

def conv_vel_a_m(dic)-> dict:
    lista=['RVOT Vmax','AV Vmax','LVOT Vmax','TR Vmax','MV Vel E','MV Vel A']
    for key in dic.keys():
        if key in lista:
            if isinstance(dic.get(key),list):
                vel=[]
                for i in dic.get(key):
                    vel.append(round(i/100,2))
                dic[key]=vel
            else:
                dic[key]=round(dic.get(key)/100,2)
    return dic
 